#### pytorch fashionmnist数据集 神经网络搭建和训练

In [14]:
# 导入必要包
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms.v2 import ToTensor     # 转换图像数据为张量
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader  # 数据加载器

In [26]:
# 定义超参数
epochs = 10
BATCH_SIZE = 128
MODEL_PATH = "fashion_mnist_model.pth"

In [28]:
class FashionMNISTModel(nn.Module):
    # 定义模型
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        # 定义模型结构
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.Sigmoid(),
            nn.Linear(512, 256),
        )
    # 定义前向传播
    def forward(self, x):
        return self.model(x)

In [29]:
# 定义训练函数
def load_data():
    train_data = FashionMNIST(root='./fashion_data', train=True, download=True, transform=ToTensor())
    test_data = FashionMNIST(root='./fashion_data', train=False, download=True, transform=ToTensor())
    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

In [30]:
def train_model(model, train_loader, epochs, lr):
    # 定义损失函数和优化器
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    # 训练模型
    for epoch in range(epochs):
        for data, target in train_loader:
            output = model(data.view(-1, 784))
            loss = loss_fn(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    # 保存模型
    torch.save(model.state_dict(), MODEL_PATH)
    print("Model saved successfully.")

In [31]:
def test_model(model, test_loader):
    # 测试模型
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    # 不计算梯度
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.view(-1, 784))
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    return 100* correct / total

In [32]:
def main():
    # 加载数据
    train_loader, test_loader = load_data()
    # 定义学习率
    learning_rates = [0.01, 0.001, 0.0001]
    # 不同学习率下的正确率
    for lr in learning_rates:
        print(f"Training with learning rate: {lr}")
        model = FashionMNISTModel()
        train_model(model, train_loader, epochs, lr)
        accuracy = test_model(model, test_loader)
        print(f"Test Accuracy with learning rate {lr}: {accuracy:.2f}%\n")

In [33]:
main()

Training with learning rate: 0.01
Epoch 1/10, Loss: 1.8664
Epoch 2/10, Loss: 1.3798
Epoch 3/10, Loss: 1.2025
Epoch 4/10, Loss: 0.9790
Epoch 5/10, Loss: 0.8704
Epoch 6/10, Loss: 0.8807
Epoch 7/10, Loss: 0.7674
Epoch 8/10, Loss: 0.7147
Epoch 9/10, Loss: 0.7016
Epoch 10/10, Loss: 0.6967
Model saved successfully.
Test Accuracy with learning rate 0.01: 75.05%

Training with learning rate: 0.001
Epoch 1/10, Loss: 2.5761
Epoch 2/10, Loss: 2.3396
Epoch 3/10, Loss: 2.2280
Epoch 4/10, Loss: 2.1713
Epoch 5/10, Loss: 2.1344
Epoch 6/10, Loss: 2.0720
Epoch 7/10, Loss: 2.0195
Epoch 8/10, Loss: 1.9652
Epoch 9/10, Loss: 1.8604
Epoch 10/10, Loss: 1.8664
Model saved successfully.
Test Accuracy with learning rate 0.001: 66.01%

Training with learning rate: 0.0001
Epoch 1/10, Loss: 5.0364
Epoch 2/10, Loss: 4.5209
Epoch 3/10, Loss: 3.9876
Epoch 4/10, Loss: 3.6064
Epoch 5/10, Loss: 3.3044
Epoch 6/10, Loss: 3.0655
Epoch 7/10, Loss: 2.8871
Epoch 8/10, Loss: 2.7594
Epoch 9/10, Loss: 2.6693
Epoch 10/10, Loss: 2.